# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотеки pandas
import pandas as pd
# импортируем функцию mystem из библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem() 
# чтение файла с данными и сохранение в df
try:
    df = pd.read_csv('F:\datasets\data.csv') # локальный путь
except:
    df = pd.read_csv('/x/x.csv') # серверный путь
df.head(10) # получение первых 10 строк таблицы df



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info() # получение общей информации о df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# количество пропущенных значений в колонке days_employed
df.days_employed.isna().sum()

2174

In [4]:
# количество пропущенных значений в колонке total_income
df.total_income.isna().sum()

2174

**Вывод**

В таблице 12 столбцов. 

Согласно документации к данным:
* `children` — количество детей в семье. Тип данных в столбце - `int64`;
* `days_employed` — общий трудовой стаж в днях. Тип данных в столбце - `float64`;  
* `dob_years` — возраст клиента в годах. Тип данных в столбце - `int64`;
* `education` — уровень образования клиента. Тип данных в столбце - `object`;
* `education_id` — идентификатор уровня образования. Тип данных в столбце - `int64`;
* `family_status ` — семейное положение. Тип данных в столбце - `object`;
* `family_status_id ` — идентификатор семейного положения. Тип данных в столбце - `int64`;
* `gender ` — пол клиента. Тип данных в столбце - `object`;
* `income_type` — тип занятости. Тип данных в столбце - `object`;
* `debt` — имел ли задолженность по возврату кредитов. Тип данных в столбце - `int64`;
* `total_income` — ежемесячный доход.Тип данных в столбце - `float64;
* `purpose` — цель получения кредит. Тип данных в столбце - `object`.

Выводы которые можно сделать после получения общей информации по датафрейму :

1. В столбцах `days_employed` и `total_income` присутствуют пропущенные в значения. Количество пропущенных значений в данных столбцах одинаково - 2174.

2. В столбце `days_employed` присутствуют отрицательные значения что может свидетельствовать о некоректной выгрузки базы данных.

3. В столбце `education` можно заметить присутствие неявных дубликатов.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# выведем первые строки с пропусками по колонке days_employed
df[df['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [6]:
# выведем первые строки с пропусками по колонке total_income
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Строки, в котроых произошел пропуск и их количество полностью совподают у двух  колонок `days_employed` и `total_income`

Так как значеня в колонках являются количественными переменными и есть определенная зависимость от категориальной переменной расположенной в колонке `income_type` рассчитаем значения медиан для каждой категории. После заполним пропущенные значения медианой в соответствии с категорией в колонке `income_type` 

In [7]:
# Сгруппировав по колонке 'income_type' надйдем медианы по колонке 'days_employed' и сохраним в переменной days_employed_median
days_employed_median = df.groupby('income_type')['days_employed'].median()

In [8]:
# Сгруппировав по колонке 'income_type' надйдем медианы по колонке 'total_income' и сохраним в переменной total_income_median
total_income_median = df.groupby('income_type')['total_income'].median()

In [9]:
'''Используя days_employed_median заполним заполним пропущенные значения в 'days_employed' == value 
   в зависимости от значения в колонке 'income_type' == key '''
for key, value in days_employed_median.items():
    df.loc[df['income_type'] == key, 'days_employed'] = df.loc[df['income_type'] == key, 'days_employed'].fillna(value)

In [10]:
'''Используя total_income_median заполним заполним пропущенные значения в 'total_income' == value 
   в зависимости от значения в колонке 'income_type' == key '''
for key, value in total_income_median.items():
    df.loc[df['income_type'] == key, 'total_income'] = df.loc[df['income_type'] == key, 'total_income'].fillna(value)

In [11]:
#Проверим что пропусков больше нет
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Используя метод **median()** и цикл **for**  заполнили все пропущенные значения медианными в колонках `days_employed` и `total_income` в зависимости от типазанятости заемщика.

### Замена типа данных

Для перевода вещественного типа данных в колонках `days_employed` и `total_income` на целочисленное значение подойдет метод **astype()**  с аргументом 'int'. 
Для перевода в положительное значение будем применять функцию **abs()**.

In [12]:
# Замена типа данных в колонке days_employed
df['days_employed'] = df['days_employed'].astype('int').abs()

In [13]:
# Замена типа данных в колонке total_income
df['total_income'] = df['total_income'].astype('int').abs()

In [14]:
# Проверим что тип данных поменялся на необходимый
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [15]:
# Вызовем первые 5 строк для чтобы визуально удовстовериться в изменениях.
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Используя метод **astype()** и функцию **abs()** перевели значения в колонках `days_employed` и `total_income` в необходимый тип **int**.

### Обработка дубликатов

Используя метод **value_count()**  проверим столбцы на наличие дубликатов и 'странных значений'

In [16]:
# Применим метод value_counts() к колонке 'children'
df.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В колонки присутствует скорее всего одно некоректное значение 20 и одно неверное -1

In [17]:
#Проанализируем строки в которых в столбце 'children' указано значение -1
df[df['children'] == -1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,4417,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816,профильное образование
705,-1,902,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882,приобретение автомобиля
742,-1,3174,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268,дополнительное образование
800,-1,349987,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293,дополнительное образование
941,-1,365213,57,Среднее,1,женат / замужем,0,F,пенсионер,0,118514,на покупку своего автомобиля
1363,-1,1195,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550,профильное образование
1929,-1,1461,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121,покупка жилья
2073,-1,2539,42,среднее,1,в разводе,3,F,компаньон,0,162638,покупка жилья
3814,-1,3045,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892,на проведение свадьбы
4201,-1,901,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375,операции со своей недвижимостью


In [18]:
#Проанализируем строки в которых в столбце 'children' указано значение 20
df[df['children'] == 20].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2941,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля


In [19]:
# удалим строки в которых в колонке children значения -1 и 20
df = df.loc[(df['children'] >= 0) & (df['children'] != 20)]

Возможно эти значения возникли в результати ошибки при выгрузки данных и число -1  является значением 1 а 20 значением 2.
Так как по структуре данных не прослеживается четкая зависимость этих значений от значений в других колонках и их общее количество мало в соотлношении со всем датафеймом возможно стоит удалить строки с этими значениями и провести анализ уже без них. Без этих значений сильной корректировки ответов на вопросы не произойдет.  

In [20]:
df.dob_years.unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [21]:
# Применим метод value_counts() к колонке 'dob_years'
df.dob_years.value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [22]:
#Проанализируем строки в которых в столбце 'dob_years' указано значение 0
df[df['dob_years'] == 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,2664,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,1872,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,397856,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,1158,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль


В колонке `dob_years` - возраст клиента в годах встречается значение - 0. Так как значение этого столбца в виду их малого количества не влияют на поставленные перед нами вопросы можно оствить строки с этими значеними но сообщить о проблеме с данными в этой колонке. 

In [23]:
# Применим метод value_counts() к колонке 'dob_years'
df.education.value_counts()

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В столбце `education` встречаются одинаковые значения которые отличаются регистром записи букв. Приведем все значения к нижнему регистру с помощью метода **lower()** .

In [24]:
# Приведем все значения колонки 'education' к нижнему регистру с помощью метода lower()
df.education = df.education.str.lower()

In [25]:
#проверим что дубликатов больше нет.
df.education.value_counts()

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

In [26]:
# Применим метод value_counts() к колонке 'education_id'
df.education_id.unique()

array([0, 1, 2, 3, 4])

In [27]:
# Проверим что значению в колонке 'education' соответствует только единственное значение 'education_id' 
df.groupby(['education']).education_id.unique()

education
высшее                 [0]
начальное              [3]
неоконченное высшее    [2]
среднее                [1]
ученая степень         [4]
Name: education_id, dtype: object

In [28]:
# Применим метод value_counts() к колонке 'family_status'
df['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4160
Не женат / не замужем     2799
в разводе                 1189
вдовец / вдова             952
Name: family_status, dtype: int64

In [29]:
# Применим метод value_counts() к колонке 'family_status_id'
df.family_status_id.unique()

array([0, 1, 2, 3, 4])

In [30]:
# Проверим что значению в колонке 'family_status' соответствует только единственное значение 'family_status_id' 
df.groupby(['family_status']).family_status_id.unique()

family_status
Не женат / не замужем    [4]
в разводе                [3]
вдовец / вдова           [2]
гражданский брак         [1]
женат / замужем          [0]
Name: family_status_id, dtype: object

In [31]:
# Применим метод value_counts() к колонке 'gender'
df['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

Кроме предсказуемых значени F и M в одной из строк встречается некорректное значенин 'XNA'

In [32]:
# Проанализируем строку в которой встрчается значение 'XNA'
display(df[df['gender'] == 'XNA'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [33]:
# удалим строки в которых в колонке gender == 'XNA'
df = df.loc[df['gender'] != 'XNA']

In [34]:
# Применим метод value_counts() к колонке 'debt'
df['debt'].unique()

array([0, 1])

In [35]:
# Применим метод value_counts() к колонке 'purpose'
df['purpose'].value_counts()

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка своего жилья                      619
покупка недвижимости                      618
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

In [36]:
#Используем метод duplicate для определения полностью совпадающих строк.
df.duplicated().sum()

71

**Вывод**

Для поиска дубликатов был использован метод **value_count()** по следующим причинам:
 - можно сразу увидеть все уникальные значения которые встречаются в колонки и их количество
 - при использовании функции lower() выводится список пар "значение-частота" который легче анализировать

По результатам использования метода **value_count()**  можно сделать следующие выводы:
 * в колонке `children` встречаются некорректные значения -1 и 20. Удалили строки с этими значениями.
 * в колонке `dob_years`  в 101 строке присутствует значение 0 чего не должно быть
 * В колонке `education`  перевели все значения в нижний регистр.
 * В колонке `gender` пристуствовало значение XNA. Удалили строку с ним.

Метод duplicate показал полное совпадение по 71 строке. Но полное совпадение строки не означает что эти данные приходятся одному человеку так как среди колонок нет значений с уникальным id клиента. В связи с этим эти строки не были удалены из датафрейма. Остальные недочеты по колонка не должны вносить серьезных коррективов при ответе на интересующие нас вопросы. 
Возможно стоило удалить строки где количество детей равно -1 и 20.

### Лемматизация

In [37]:
'''Цикл проходит по значениям списка purpose_unique, который состоит из уникальных значений колонки purpose
   и создает словарь purpose_dict в котором ключом является уникальное значение из колоки purpose
   а значением лемматизированное строка этой же колонки
'''
purpose_unique = df['purpose'].unique()
purpose_dict = {}
for i in purpose_unique:
    lemmas = m.lemmatize(i)
    purpose_dict[i] = lemmas
    

**Вывод**

Создали словарь с лемматизированными значениями колонки puprose для более удобной категоризации по целям кредита

### Категоризация данных

In [38]:
''' функция purpose_id  принимает значение из колонки purpose,
    исползует его как ключ для получения соответствующего списка  в словаре purpose_dict
    и далее с помощью логического ветвления, возвращает числовое значение 
    вновь создоваеммной колонки purpose_id, в которой происходит числовая категоризация целей кредита.
'''
def purpose_id(purpose):
    purpose_list = purpose_dict[purpose]
    if 'автомобиль' in purpose_list:
        return 'автомобиль'
    if 'свадьба' in purpose_list:
        return 'свадьба'
    if 'образование' in purpose_list:  
        return 'образование'
    else:
        return 'недвижимость'
df['purpose_lemma'] = df['purpose'].apply(purpose_id)

In [39]:
# выведем первые 10 значений
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


In [40]:
# какая цель кредита самая популярная среди клиентов
df['purpose_lemma'].value_counts()

недвижимость    10779
автомобиль       4288
образование      3997
свадьба          2337
Name: purpose_lemma, dtype: int64

Категоризируем колнку ежемесячного дохода `total_income` разделив её на 3 условные группы:
 * низший класс
 * средний класс
 * высший класс

In [41]:
# Неайдем медиану среди значений ежемесячного дохода клиентов
middle = df['total_income'].median()
middle

142594.0

In [42]:
# Найдем  границу по которой будет происходить категоризация на низший и средний классы
df[df['total_income'] < 142594]['total_income'].median()

103525.5

In [43]:
# Найдем  границу по которой будет происходить категоризация на средний и высший классы
df[df['total_income'] > 142594]['total_income'].median()

196554.0

In [44]:
def income_group(total_income):
    ''' Возвращает группу по значению total_income, используя правила:
     - 'низший класс', если total_income <= 103532 лет;
     - 'средний класс', если total_income от 103533 до 196542;
     - 'высший класс' — от 196543 и старше
    '''
    if total_income <= 103532:
        return 'низший класс'
    if total_income <= 196542:
        return 'средний класс'
    return 'высший класс'

#добавим колонку income_group  используя функцию income_group.
df['income_group'] = df['total_income'].apply(income_group)

In [45]:
# выведем первые 10 значений
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высший класс
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний класс
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,средний класс
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высший класс
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,средний класс
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высший класс
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высший класс
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний класс
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,низший класс
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,средний класс


In [46]:
# какая цель кредита самая популярная среди клиентов
df['income_group'].value_counts()

средний класс    11245
высший класс      5280
низший класс      4876
Name: income_group, dtype: int64

**Вывод**

Произведя категоризацию клиентов относительно цели получения кредита можно сделать вывод что половина из них берут кредит на недвижимость а меньше всего на свадьбу.

Распределили клиентов относительно ежемесячного дохода на 3 группы для дальнейшего анализа:
 * клиенты с ежемесечным доходом меньше 103533 попадают в категорию "низший класс" 
 * клиенты с ежемесечным доходом от 103533 до 196542 попадают в категорию "средний класс" 
 * клиенты с ежемесечным доходом меньше 196543 попадают в категорию "высший класс" 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [47]:
'''воспользуемся методами pivot_table, sort_values и round()
   pivot_table - метод для создания сводной таблицы 
   sort_values - сортировка по убыванию значения в колонке debt
   round() - оставим 3 знака после после "запятой" для значений.
''' 
df.pivot_table(index = ['children'], values = 'debt' ).sort_values(by  = 'debt', ascending=False).round(3)

,debt
children,
4,0.098
2,0.094
1,0.092
3,0.082
0,0.075
5,0.000


**Вывод**

Можно сказать что клиенты у которых нет детей имеют в среднем на 1.5% меньше задолжности относительно клиентов с детьми. Больше всего доля должников среди людей у которых 4 ребенка. Так как количество клиентов с пятью детьми слишком мало их можно не учитывать. 


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
'''воспользуемся методами pivot_table, sort_values и round()
   pivot_table - метод для создания сводной таблицы 
   sort_values - сортировка по убыванию значения в колонке debt
   round() - оставим 3 знака после после "запятой" для значений.
''' 
df.pivot_table(index = ['family_status'], values = 'debt' ).sort_values(by  = 'debt', ascending=False).round(3)

,debt
family_status,
Не женат / не замужем,0.098
гражданский брак,0.093
женат / замужем,0.075
в разводе,0.071
вдовец / вдова,0.066


**Вывод**

Меньший процент задолжности имеют клиенты в статусе семейного положения вдовец / вдова - 6.5
Максимальный процент задолжности имеют клиенты которые не были женаты или за мужем - 9.74

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
df.pivot_table(index = ['income_group'], values = 'debt' ).sort_values(by  = 'debt', ascending=False).round(3)

,debt
income_group,
средний класс,0.086
низший класс,0.079
высший класс,0.071


**Вывод**

Наименьший процент задолжности имеют клиенты котрые относятся к группе "высший класс" - 7.1, а
максимальный клиенты котрые относятся к группе "средний класс" - 8.7

- Как разные цели кредита влияют на его возврат в срок?

In [50]:
df.pivot_table(index = ['purpose_lemma'], values = 'debt').sort_values(by  = 'debt', ascending=False).round(3)

,debt
purpose_lemma,
автомобиль,0.093
образование,0.092
свадьба,0.078
недвижимость,0.072


**Вывод**

Минимальный процент задолжности по кредиту имеют клиенты которые брали его на решение жилищного вопроса и проведение свадьбы - 7.2 и 7.9 соответственно.
Клиенты которые брали кредит на автомобиль имеют максимальный процент задолжности - 9.3


## Шаг 4. Общий вывод

В результате прдобработки данных можно сделать следующие выводы:
 * в колонке `days_employed` и `total_income` были обнаружены пропуски значений в одних и тех же строках. Пропуски не зависили от значений в других колонках и скорее всего связаны
 * заполнили колонки с пропусками значениями медианы от группировки по колонки `income_type`
 * в колонке `children` встречаются некорректные значения -1 и 20 
 * в колонке `dob_years`  присутствует значение некорректное значение - 0. Количество таких строк - 101
 * В колонке `education`  перевели все значения в нижний регистр.
 * В колонке `gender` пристуствовало значение XNA котторое было заменено на значение M
 * Лемматиировали колонку `purpose` для дальнейшей категоризации по целям кредита
 * Произвели категоризацию по целям кредита и ежемесячному доходу использую для этого функции.

После проведения всех шагов по предобработки данных были получены слудущие ответы на поставленные вопросы:
 * Клинты у которых нет детей имеют меньший процент задолжности относитеьно клиентов с детьми. Так как выборка из пяти детей слишком маленькая делать на основании нее выводы некорректно.
 * Максимальный процент задолжности имеют клиенты которые не были женаты или за мужем - 9.8. Меньший процент задолжности имеют клиенты в статусе семейного положения вдовец / вдова - 6.6
 * Наименьший процент задолжности имеют клиенты котрые относятся к группе "высший класс" - 7.1, а максимальный клиенты ,относящиеся к группе "средний класс" - 8.6.
 * Минимальный процент задолжности по кредиту имеют клиенты которые брали его на решение жилищного вопроса и проведение свадьбы - 7.2 и 7.8 соответственно. Клиенты которые брали кредит на автомобиль имеют максимальный процент задолжности - 9.3
 